In [1]:
import requests
import os.path
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Load dataset

In [2]:
if not os.path.isfile('../data/voting_behavior_V2_19492021.dta'):
    print('Downloading voting_behavior dataset...')
    with open('../data/voting_behavior_V2_19492021.dta', 'wb') as out_file:
        content = requests.get('https://dataverse.harvard.edu/api/access/datafile/6402445', stream=True).content
        out_file.write(content)

voting_beh = pd.read_stata('../data/voting_behavior_V2_19492021.dta')

# Static Preprocessing

**Note:** "Data Enrichment", so the handling of missing values will be dealt with in the next notebook. We will experiment with multiple approaches, which is why we shift this step in the classification pipeline.  

### Data Reduction

- time span of votes
- irrelevant and duplicate columns
- MPs that don't belong to a party 

In [5]:
# including votes only from the last election period
reduced = voting_beh.loc[voting_beh['vote_date'] > '2017-10-24']

# reducing columns (specify columns to keep)
reduced = reduced[['id_de_parliament', 'party_text', 'vote_id', 'vote_beh']]

# renaming 'id_de_parliament' to 'mp_id' and 'party_text' to 'party'
reduced = reduced.rename({'id_de_parliament':'mp_id', 'party_text':'party'}, axis=1)

# There are four MPs that don't belong to a party
# Remove them because they won't help in predicting the party based on votes
reduced = reduced[reduced['party']!='']

# Merge 'CDU' and 'CSU' to 'CDU/CSU'
reduced.loc[reduced['party'].isin(['CDU','CSU']), 'party'] = 'CDU/CSU'

display(reduced)

,mp_id,party,vote_id,vote_beh
320229,11001938.0,CDU/CSU,19022.0,yes
320230,11001938.0,CDU/CSU,19232.0,yes
320234,11001938.0,CDU/CSU,19146.0,no
320237,11001938.0,CDU/CSU,19177.0,yes
320251,11001938.0,CDU/CSU,19065.0,unexcused absent
...,...,...,...,...
1435244,11004971.0,AfD,19242.0,yes
1435245,11004971.0,AfD,19243.0,no
1435246,11004972.0,Linke,19243.0,no
1435247,11004972.0,Linke,19242.0,no


### Data Transformation

- renaming of attributes
- pivoting (unstacking long table format to wide)
- adjust the datatypes

In [9]:
transformed = reduced.copy()

# set the datatype of the ID from float64 to int (int64)
transformed[['mp_id']] = transformed[['mp_id']].astype(int)
transformed[['vote_id']] = transformed[['vote_id']].astype(int)

# pivots the dataset so that there is only one sample for each MP with attributes for each vote holding the respective voting behaviour
transformed = transformed.pivot(index=['mp_id', 'party'], columns=['vote_id'], values=['vote_beh'])

# reset the index to be able to access data as usually
transformed = transformed.reset_index()

# remove mp_id (not necessary for classification)
transformed = transformed.drop('mp_id', axis=1)

# make column index flat / non-hierarchical and format name
transformed.columns = transformed.columns.to_flat_index()
transformed.columns = transformed.columns.map(lambda x: str(x).replace('(', ''))
transformed.columns = transformed.columns.map(lambda x: str(x).replace('\'', ''))
transformed.columns = transformed.columns.map(lambda x: str(x).replace(', )', ''))
transformed.columns = transformed.columns.map(lambda x: str(x).replace('beh, ', ''))
transformed.columns = transformed.columns.map(lambda x: str(x).replace(')', ''))

display(transformed)

C:\Users\mogra\AppData\Local\Temp\ipykernel_3328\2745563609.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  transformed = transformed.drop('mp_id', axis=1)


,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,yes
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,yes,yes,yes,yes,yes,yes,no,excused absence,excused absence,unexcused absent
2,Linke,no,no,no,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,abstain,no,no
3,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,excused absence
4,Linke,excused absence,excused absence,excused absence,excused absence,excused absence,unexcused absent,unexcused absent,unexcused absent,excused absence,...,no,no,no,no,no,no,no,abstain,no,excused absence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,FDP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,no,no,yes,yes,no,no,yes,no,no
741,GRÜNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,yes,no,no,yes,yes,yes,no,yes,no,no
742,CDU/CSU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes
743,AfD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,no


In [15]:
# validating transformed dataframe
print('Correct amount of minister president: ', len(transformed) == reduced['mp_id'].nunique())
print('Correct amount of votes: ', len(transformed.columns) - 1 == reduced['vote_id'].nunique())
print('Correct set of parties: ', transformed['party'].unique())

Correct amount of minister president:  True
Correct amount of votes:  True
Correct set of parties:  ['CDU/CSU' 'SPD' 'Linke' 'FDP' 'GRÜNE' 'AfD']


### Create alternatives

*Note:* For Imputing, we try out both, the vote categories 'unexcused absent' and 'excused absence' separate and merged together to 'absence'. That is why we need to create two different datasets.

In [10]:
# 'unexcused absent' and 'excused absence' as separate values
transformed_alternative = transformed.copy()

# Merge the 'unexcused absent' and 'excused absence' to 'absence'
transformed.replace(['unexcused absent','excused absence'], 'absence', inplace=True)

### Save files

In [11]:
# export as pickle files
transformed_alternative.to_pickle('../data/btvote_alternative.pkl')
transformed.to_pickle('../data/btvote.pkl')